# Stan Model Building Workflow

When writing a Stan model, as when writing any other computer program,\
*the fastest way to success is to go slowly.*

* Incremental development
   + Write a (simple) model
   + Fit the model to data (either simulated or observed)
   + Check the fit 

* Then modify *(stepwise)* and repeat

* Compare successive models

## Notebook Setup

In [ ]:
# import all libraries used in this notebook
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import libpysal as sa
import matplotlib
import splot as splt
import plotnine as p9
import arviz as az
%matplotlib inline

from cmdstanpy import CmdStanModel, cmdstan_path, cmdstan_version

# suppress plotnine warnings
import warnings
warnings.filterwarnings('ignore')

# setup plotnine look and feel
p9.theme_set(
  p9.theme_grey() + 
  p9.theme(text=p9.element_text(size=10),
        plot_title=p9.element_text(size=14),
        axis_title_x=p9.element_text(size=12),
        axis_title_y=p9.element_text(size=12),
        axis_text_x=p9.element_text(size=8),
        axis_text_y=p9.element_text(size=8)
       )
)
xlabels_90 = p9.theme(axis_text_x = p9.element_text(angle=90, hjust=1))

map_theme =  p9.theme(figure_size=(7,6),
                 axis_text_x=p9.element_blank(),
                 axis_ticks_x=p9.element_blank(),
                 axis_text_y=p9.element_blank(),
                 axis_ticks_y=p9.element_blank())

#### Load the NYC study data

In [ ]:
nyc_geodata = gpd.read_file(os.path.join('data', 'nyc_study.geojson'))
print(nyc_geodata.columns)
print(nyc_geodata['BoroName'].value_counts())

#### Assemble the input data

The data block of the model declares variables:

- `N` - the number of census tracts
- `y` - the array of observed outcomes - accidents per tract
- `E` - the population per tract ("exposure")
- `K` - the number of predictors
- `xs` - the N x K data matrix of predictors

The predictors from the study data are columns: `pct_pubtransit`,`med_hh_inc`, `traffic`, `frag_index`.

**Python**

In [ ]:
design_vars = np.array(['pct_pubtransit','med_hh_inc', 'traffic', 'frag_index'])

design_mat = nyc_geodata[design_vars].to_numpy()
design_mat[:, 1] = np.log(design_mat[:, 1])
design_mat[:, 2] = np.log(design_mat[:, 2])

pois_data = {"N":nyc_geodata.shape[0],
             "y":nyc_geodata['count'].astype('int'),
             "E":nyc_geodata['kid_pop'].astype('int'),
             "K":4,
             "xs":design_mat }

## Base Model:  `poisson.stan`

This file is in directory `stan/poisson.stan`.

In [ ]:
poisson_model_file = os.path.join('stan', 'poisson.stan')

with open(poisson_model_file, 'r') as file:
    contents = file.read()
    print(contents)

#### Model is compiled (as needed) on instantiation

In [ ]:
pois_mod = CmdStanModel(stan_file=poisson_model_file)

#### Run the NUTS-HMC sampler

In [ ]:
pois_fit = pois_mod.sample(data=pois_data)

#### Summarize the results

In [ ]:
pois_fit_summary = pois_fit.summary()
pois_fit_summary.round(2).loc[
  ['beta0', 'betas[1]', 'betas[2]', 'betas[3]', 'betas[4]']]

## Refinement: Mean-Center Predictor Data

*In theory*, much discussion and debate about doing this -
start
[here](https://www.goldsteinepi.com/blog/thewhyandwhenofcenteringcontinuouspredictorsinregressionmodeling/index.html),
follow the links and keep going.

*In practice*, ***this helps, alot!*** and is often necessary in order to fit the model.
The BRMS package centers continuous data values on zero - [discussion here](https://discourse.mc-stan.org/t/brms-input-scaling-clarification/23601/3).

Doing this requires two additions to the model

1. In the `transformed data` block, compute the mean of a data column, then subtract the mean from the column.

2. If the regression has an intercept term, in the `generated quantities` block, adjust for this by adding back the dot-product of the mean values of each column and the regression coefficient vector to it.

There are the key changes / additions to the base model 

```stan
data {
  // no change 
}
transformed data {
  // center continuous predictors 
  vector[K] means_xs;  // column means of xs before centering
  matrix[N, K] xs_centered;  // centered version of xs
  for (k in 1:K) {
    means_xs[k] = mean(xs[, k]);
    xs_centered[, k] = xs[, k] - means_xs[k];
  }
}
parameters {
  // no change 
}
model {
  y ~ poisson_log(log_E + beta0 + xs_centered * betas);   // centered data
  // priors same
}
generated quantities {
  real beta_intercept = beta0 - dot_product(means_xs, betas);  // adjust intercept
  // compute log_lik, y_rep 
  {
    vector[N] eta = log_E + beta0 + xs_centered * betas;   // centered data
    //  ..
  }
}
```

In [ ]:
pois_xc_mod = CmdStanModel(stan_file=os.path.join(
  'stan', 'poisson_ctr_preds.stan'))

In [ ]:
pois_xc_fit = pois_xc_mod.sample(data=pois_data)  

In [ ]:
pois_xc_summary = pois_xc_fit.summary()
pois_xc_summary.round(2).loc[
  ['beta_intercept', 'beta0', 'betas[1]', 'betas[2]', 'betas[3]', 'betas[4]']]

How do the samples differ?

In [ ]:
print("data matrix original scales")   
pois_fit_summary.round(2).loc[
  ['beta0', 'betas[1]', 'betas[2]', 'betas[3]', 'betas[4]']]

## Predictor Data Scales

The data variables are on different scales

| Measures                                              | Median | Min    | Mean     | Max       |
|-------------------------------------------------------|--------|--------|----------|-----------|
| Med. household income in USD, 2010-14                 | \$53,890| \$9,327 | \$58,497  | \$232,266  |
| Pct. commute by walk/cycle/public trans, 2010-14      | 73.9   | 9.7    | 69.8     | 100.0     |
| Standardized social fragmentation index               | -0.1   | -6.7   | 0.0      | 18.7      |
| Traffic Volume (AADT), 2015                           | 19,178 | 843    | 37,248   | 276,476   |

In the previous steps, the predictor variables `med_hh_inc` and `traffic` were log-transformed.
What happens if we just use the raw data values?

In [ ]:
design_vars = np.array(['pct_pubtransit','med_hh_inc', 'traffic', 'frag_index'])
design_mat_2 = nyc_geodata[design_vars].to_numpy()

pois_data_2 = {"N":nyc_geodata.shape[0],
             "y":nyc_geodata['count'].astype('int'),
             "E":nyc_geodata['kid_pop'].astype('int'),
             "K":4,
             "xs":design_mat_2 }

print("log income, traffic\n", pd.DataFrame(design_mat).describe())
print("raw data\n", pd.DataFrame(design_mat_2).describe())

Run the base model on this data.

In [ ]:
pois_fit_2 = pois_mod.sample(data=pois_data_2)

In [ ]:
pois_summary_2 = pois_fit_2.summary()
pois_summary_2.round(2).loc[
  ['beta0', 'betas[1]', 'betas[2]', 'betas[3]', 'betas[4]']]

The model that transforms the data matrix fails for the same fundamental reason:
the regression component `xs * betas` or `xs_centered * betas` cannot be computed,
it either overflows or underflow because the predictors are on wildly different scales.
(Transforming the data fails faster because it both both `traffic` and `med_hh_inc`
include large negative values - this underflows consistently.)

In [ ]:
# uncomment next line - sampler will fail
# pois_xc_mod.sample(data=pois_data_2)

#### Takeaway

To improve model fit and processing speed, predictors should be on the same scale.

*But* remember to account for any rescaling when trying to interpret the fitted coefficients! 

## Model Checking:  The Posterior Predictive Check 

From the [Stan Users Guide](https://mc-stan.org/docs/stan-users-guide/posterior-predictive-checks.html#simulating-from-the-posterior-predictive-distribution)

>Posterior predictive checks are a way of measuring whether a model
does a good job of capturing relevant aspects of the data, such as
means, standard deviations, and quantiles.

>The posterior predictive distribution is the distribution over new
observations given previous observations.  It's predictive in the
sense that it's predicting behavior on new data that is not part of
the training set.  It's posterior in that everything is conditioned on
observed data $y$.

>The posterior predictive distribution for replications
$y^{\textrm{rep}}$ of the original data set $y$ given model parameters
$\theta$ is defined by
$$
p(y^{\textrm{rep}} \mid y)
= \int p(y^{\textrm{rep}} \mid \theta)
       \cdot p(\theta \mid y) \, \textrm{d}\theta.
$$


#### Posterior predictive density overlay plots

The variable `y_rep` is a *simulated* dataset based on the observed data and the estimated parameters, using Stan's PRNG function `poisson_log_rng`,
thus the sample contains a set of random datasets all conditioned on the observed data (nyc_geodata['count']).

The posterior predictive density overlay plot compares the distribution of the observed data to the distribution of some of the simulated datasets (`y_rep`).

In R, this is available from the [bayesplot](https://mc-stan.org/bayesplot) package as function [`ppc_dens_overlay`](https://mc-stan.org/bayesplot/articles/graphical-ppcs.html#ppc_dens_overlay).

To create the equivalent plot in Python:

1. select a random set of draws from the output.
2. create a ggplot plot
3. add  to the plot the density each simulated dataset  (thin line, light color, relatively transparent)
4. add to the plot the density of the observed data (heavy line, dark color)

In [ ]:
y_rep = pois_xc_fit.stan_variable('y_rep')

# select random sample
y_rep_sample = pd.DataFrame(y_rep).sample(n=50).reset_index(drop=True).T

# create ggplot
pois_ppc = p9.ggplot() + p9.ggtitle("Posterior Predictive Check: Poisson model") 

# add simulated dataset densities
for i in range(50):
    pois_ppc = (
        pois_ppc
        + p9.stat_density(mapping=p9.aes(x=y_rep_sample[i]),
                          geom='line', color='lightblue', alpha=0.4)
    )

# add observed data density
pois_ppc = (
    pois_ppc
    + p9.stat_density(data=nyc_geodata, mapping=p9.aes(x='count'),
                      geom='line', color='darkblue', size=1.1)
    + p9.theme(figure_size=(6,4))
)
pois_ppc

## Refinement:  Add random effects

The Poisson distribution provides a single parameter $\lambda$, which is both mean and variance.
As the above plots show, the data is overdispersed - the observed variance is greater than expected.
To improve the model fit, we can add an ordinary random-effects component - this will account for per-tract heterogeneity.
(Not to get head of ourselves, but this is one component in the BYM model).

There are the key changes / additions to the base model 
 
```stan
data {
  // no change 
}
transformed data {
  // no change, (center continuous predictors)
}
parameters {
  real beta0; // intercept
  vector[K] betas; // covariates
  vector[N] theta; // heterogeneous random effects
  real<lower=0> sigma; // random effects variance 
}
model {
  y ~ poisson_log(log_E + beta0 + xs_centered * betas + theta * sigma);
  beta0 ~ std_normal();
  betas ~ std_normal();
  theta ~ std_normal();
  sigma ~ normal(0, 5);
}
generated quantities {
  // compute log_lik, y_rep 
  {
    vector[N] eta = log_E + beta0 + xs_centered * betas + theta * sigma;
    // ..
  }
}

```


**Python**

In [ ]:
pois_re_mod = CmdStanModel(stan_file=os.path.join(
  'stan', 'poisson_re.stan'))
pois_re_fit = pois_re_mod.sample(data=pois_data)
pois_re_summary = pois_re_fit.summary()
pois_re_summary.round(2).loc[
  ['beta_intercept', 'beta0', 'betas[1]', 'betas[2]', 'betas[3]', 'betas[4]', 'sigma']]

How do the samples differ from the base model?

In [ ]:
pois_xc_summary.round(2).loc[
  ['beta_intercept', 'beta0', 'betas[1]', 'betas[2]', 'betas[3]', 'betas[4]']]

Run the PPC plots

In [ ]:
y_rep_re = pois_re_fit.stan_variable('y_rep')

# select random sample
y_rep_re_sample = pd.DataFrame(y_rep_re).sample(n=50).reset_index(drop=True).T

# create ggplot
pois_re_ppc = p9.ggplot() + p9.ggtitle("Posterior Predictive Check: Poisson + RE model") 

# add simulated dataset densities
for i in range(50):
    pois_re_ppc = (
        pois_re_ppc
        + p9.stat_density(mapping=p9.aes(x=y_rep_re_sample[i]),
                          geom='line', color='lightblue', alpha=0.4)
    )

# add observed data density
pois_re_ppc = (
    pois_re_ppc
    + p9.stat_density(data=nyc_geodata, mapping=p9.aes(x='count'),
                      geom='line', color='darkblue', size=1.1)
    + p9.theme(figure_size=(6,4))
)
pois_re_ppc

In [ ]:
pois_ppc

## Model Comparison


#### Leave-one-out cross-validation (LOO)


The [loo package](https://mc-stan.org/loo/) provides an implementation of the algorithm presented in

* Vehtari, A., Gelman, A. & Gabry, J. Practical Bayesian model evaluation using leave-one-out cross-validation and WAIC. Stat Comput 27, 1413–1432 (2017). https://doi.org/10.1007/s11222-016-9696-4
* Vehtari, A.,Simpson, D.P.,  Gelman, A., Yao, Y., & Gabry, J. Pareto Smoothed Importance Sampling, arxiv, 2024. https://arxiv.org/abs/1507.02646

Pareto-smoothed importance sampling (PSIS) provides stabilized effective sample size estimates, Monte Carlo error estimates, and convergence diagnostics and provide a way to efficiently perform
leave-one-out (LOO) cross-valiation, which allows us to compare predictive errors between two models.

In R,  we can do model comparison using the `loo` package.

In Python, we can do this using [ArviZ](https://python.arviz.org/en/stable/).

In [ ]:
## model comparison using loo from Arviz
import arviz as az
idata_pois_xc = az.from_cmdstanpy(
    pois_xc_fit,
    posterior_predictive="y_rep",
    dims={"betas": ["covariates"]},
    coords={"covariates": design_vars},
    observed_data={"y": pois_data['y']}
)
idata_pois_xc

ArviZ provides the equivalent functions of those in R packages `bayesplot` and `loo`.

In [ ]:
az.plot_ppc(idata_pois_xc, data_pairs={"y":"y_rep"})

In [ ]:
idata_pois_re = az.from_cmdstanpy(
    pois_re_fit,
    posterior_predictive="y_rep",
    dims={"betas": ["covariates"]},
    coords={"covariates": design_vars},
    observed_data={"y": pois_data['y']}
)
az.plot_ppc(idata_pois_re, data_pairs={"y":"y_rep"})

In [ ]:
az.compare({"poisson":idata_pois_xc, "poisson_re":idata_pois_re})